# 2nd Level Model Structure: Reactive Stage

In [1]:
import sys
import os
import pickle
sys.path.append(os.path.abspath('..'))
import numpy as np
from matplotlib import pyplot as plt

In [2]:
from pyomo import environ as pe
from global_sets.component import m
from utility.display_utility import trans_product_mole, trans_product_mass
from utility.model_utility import add_dual, update_dual, check_DOF

# stage construction rules
from physics.kinetics_bounded import kinetic_block_rule
from physics.energy_bounded import energy_block_rule
from physics.VLE_bounded_MPCC_P import VLE_block_rule
from physics.MPCC_T import dew_block_rule

model = pe.ConcreteModel()

# Global Sets (Inlet / Outlet)

In [3]:
model.inlet = pe.Set(initialize=['in'])
model.outlet = pe.Set(initialize=['out','P'])
model.stream = model.inlet | model.outlet

# Global Variables

In [4]:
# Tray Inlet/Outlet Variable
model.x_ = pe.Var(model.inlet,m.COMP_TOTAL,within=pe.NonNegativeReals)
model.y_ = pe.Var(model.inlet,m.COMP_TOTAL,within=pe.NonNegativeReals)
model.x = pe.Var(m.COMP_TOTAL,within=pe.NonNegativeReals)
model.y = pe.Var(m.COMP_TOTAL,within=pe.NonNegativeReals)
model.z = pe.Var(m.COMP_FEED,within=pe.NonNegativeReals)

model.L = pe.Var(model.stream,within=pe.NonNegativeReals)
model.V = pe.Var(model.stream,within=pe.NonNegativeReals)
model.F = pe.Var(within=pe.NonNegativeReals)

model.H_L_ = pe.Var(model.inlet,within=pe.Reals)
model.H_V_ = pe.Var(model.inlet,within=pe.Reals)
model.H_L = pe.Var(within=pe.Reals)
model.H_V = pe.Var(within=pe.Reals)
model.H_F = pe.Var(within=pe.Reals)

# State Variable
model.T = pe.Var(within=pe.NonNegativeReals,bounds=(200+273.15,300+273.15)) # K
model.T_F = pe.Var(within=pe.NonNegativeReals) # K
model.P = pe.Var(within=pe.NonNegativeReals,bounds=(10,30)) # Bar

model.f_V = pe.Var(m.COMP_TOTAL,within=pe.NonNegativeReals,initialize=1e-20)
model.f_L = pe.Var(m.COMP_TOTAL,within=pe.NonNegativeReals,initialize=1e-20)

model.cat = pe.Var(within=pe.NonNegativeReals) # kg
model.Q_main = pe.Var(within=pe.Reals) # MW
model.r_total_comp = pe.Var(m.COMP_TOTAL,within=pe.Reals) # kmol/s

# Construct Individual Blocks

In [5]:
model.kinetics_block = pe.Block(rule=kinetic_block_rule)

> Importing Kinetics Blocks......
> Adding the following local variable:
--------------------------------------------------
| kinetics_block.k_FT
| kinetics_block.r_FT_total
| kinetics_block.g0_FT
| kinetics_block.alpha
| kinetics_block.r_FT_cnum
| kinetics_block.r_FT_comp
| kinetics_block.k_WGS
| kinetics_block.Ke_WGS
| kinetics_block.r_WGS
| kinetics_block.r_WGS_comp
--------------------------------------------------



In [6]:
model.energy_block = pe.Block(rule=energy_block_rule)

> Importing Energy Blocks......
> Adding the following local variable:
--------------------------------------------------
| energy_block.dH_F
| energy_block.dH_V
| energy_block.dH_L
| energy_block.dH_vap
--------------------------------------------------



In [7]:
model.VLE_block = pe.Block(rule=VLE_block_rule)

> Importing VLE Blocks......
> Adding the following local variable:
--------------------------------------------------
| VLE_block.P_VLE
| VLE_block.n_ave
| VLE_block.n_ave_cal
| VLE_block.Hen
| VLE_block.Hen0
| VLE_block.gamma
| VLE_block.P_sat
| VLE_block.P_sat_Y
| VLE_block.P_sat_dY_inf
| VLE_block.P_sat_dY0
| VLE_block.Hen_ref
| VLE_block.Hen0_ref
| VLE_block.gamma_ref
| VLE_block.V_L
| VLE_block.V_L_dY_inf
| VLE_block.V_L_dY0
| VLE_block.poynting
--------------------------------------------------



# Standard MESH Equations

## Mass Balance

\begin{equation}
Fz_{i}+\sum_{s\in{inlet}}L_{s}x'_{i,s}+\sum_{s\in{inlet}}V_{s}y'_{i,s}-\sum_{s\in{outlet}}L_{s}x_{i,s}+\sum_{s\in{ouelet}}V_{s}y_{i,s}+R_{i} = 0 \\
i = 1,...NC
\end{equation}

In [8]:
def mass_balance_main_rule(model,i):
    if i in m.COMP_FEED:
        return model.F*model.z[i] + sum(model.L[s]*model.x_[s,i] + model.V[s]*model.y_[s,i] for s in model.inlet)\
        + model.r_total_comp[i] - sum(model.L[s]*model.x[i] + model.V[s]*model.y[i] for s in model.outlet) == 0
    else:
        return sum(model.L[s]*model.x_[s,i] + model.V[s]*model.y_[s,i] for s in model.inlet)\
        + model.r_total_comp[i] - sum(model.L[s]*model.x[i] + model.V[s]*model.y[i] for s in model.outlet) == 0
model.mass_balance_main_con = pe.Constraint(m.COMP_TOTAL,rule=mass_balance_main_rule)

## Equilibrium

\begin{align}
&f_{i,V} = f_{i,L} \\
&i = 1,...NC
\end{align}

In [9]:
def VL_equil_rule(model,i):
    return model.f_V[i] == model.f_L[i]
model.VL_equil_con = pe.Constraint(m.COMP_TOTAL,rule=VL_equil_rule)

In [10]:
model.MPCC = pe.Block(rule=dew_block_rule)

> Importing dew Blocks......
> Adding the following local variable:
------------------------------------
| MPCC.x
| MPCC.y
| MPCC.T
| MPCC.P
| MPCC.f_V
| MPCC.f_L
| MPCC.beta
| MPCC.s_L
| MPCC.s_V
| MPCC.epi
------------------------------------

> Importing VLE Blocks......
> Adding the following local variable:
--------------------------------------------------
| MPCC.VLE_block.n_ave
| MPCC.VLE_block.n_ave_cal
| MPCC.VLE_block.Hen
| MPCC.VLE_block.Hen0
| MPCC.VLE_block.gamma
| MPCC.VLE_block.P_sat
| MPCC.VLE_block.P_sat_Y
| MPCC.VLE_block.P_sat_dY_inf
| MPCC.VLE_block.P_sat_dY0
| MPCC.VLE_block.Hen_ref
| MPCC.VLE_block.Hen0_ref
| MPCC.VLE_block.gamma_ref
| MPCC.VLE_block.V_L
| MPCC.VLE_block.V_L_dY_inf
| MPCC.VLE_block.V_L_dY0
| MPCC.VLE_block.poynting
--------------------------------------------------



## Summation

\begin{equation}
\sum_ix_{i} = \sum_iy_{i} \\
F + L_{in} + V_{in} + R = L_{out} + V_{out}
\end{equation}

In [11]:
def summation_x_y_rule(model):
    return sum(model.x[i] for i in m.COMP_TOTAL) == sum(model.y[i] for i in m.COMP_TOTAL)
model.summation_x_y_con = pe.Constraint(rule=summation_x_y_rule)

def summation_total_mass_rule(model):
    return model.F + sum(model.L[s] + model.V[s] for s in model.inlet) + sum(model.r_total_comp[i] for i in m.COMP_TOTAL)\
            - sum(model.L[s] + model.V[s] for s in model.outlet) == 0
model.summation_total_mass_con = pe.Constraint(rule=summation_total_mass_rule)

## Energy Balance

\begin{equation}
F H_f+\sum_{s\in{inlet}}L_{s}H_{l,s}+\sum_{s\in{inlet}}V_{s}H_{v,s}-\sum_{s\in{outlet}}L_{s}H_{l,s}-\sum_{s\in{outlet}}V_{s}H_{v,s}+Q = 0
\end{equation}

In [12]:
def heat_balance_main_rule(model):
    return model.F*model.H_F + sum(model.L[s]*model.H_L_[s] + model.V[s]*model.H_V_[s] for s in model.inlet) \
            + model.Q_main - sum(model.L[s]*model.H_L + model.V[s]*model.H_V for s in model.outlet) == 0
model.heat_balance_main_con = pe.Constraint(rule=heat_balance_main_rule)

# Testing

In [13]:
add_dual(pe,model)

Created the follow pyomo suffixes:
ipopt_zL_out, ipopt_zU_out, ipopt_zL_in, ipopt_zU_in, dual


## Fixing In/Product Redundent Flow

In [14]:
model.x_.fix(0)
model.y_.fix(0)
model.L['in'].fix(0)
model.L['P'].fix(0)
model.V['in'].fix(0)
model.V['P'].fix(0)
model.H_L_.fix(0)
model.H_V_.fix(0)

## Fixing Model Parameters

In [15]:
model.cat.fix(300)
model.P.fix(20)
model.T_F.fix(200+273.15)
model.F.fix(1)
model.z['CO'].fix(1/(1+0.8)-0/2)
model.z['H2'].fix(0.8/(1+0.8)-0/2)
model.z['C30H62'].fix(0)
model.VLE_block.n_ave.fix(20)

In [16]:
check_DOF(pe,model)

Active Equality Constraints:	 1895
Active Inequality Constraints:	 2
Active Variables:		 2071
Fixed Variables:		 173
DOF:				 3


## Fix T or to fix Q? we have found that Maximize against an upper bound is most reliable

In [17]:
model.T.setub(210+273.15)

In [18]:
# model.obj = pe.Objective(expr = model.L['out'],sense=pe.maximize)
# model.obj = pe.Objective(expr = model.Q_main,sense=pe.maximize)
model.obj = pe.Objective(expr = model.T,sense=pe.maximize)

In [19]:
opt = pe.SolverFactory('ipopt')

opt.options['print_user_options'] = 'yes'
opt.options['linear_solver'] = 'ma86'
opt.options['linear_system_scaling '] = 'mc19'
opt.options['linear_scaling_on_demand '] = 'no'
# opt.options['nlp_scaling_method'] = None
# opt.options['bound_relax_factor'] = 0
# opt.options['halt_on_ampl_error'] = 'yes'

opt.options['max_iter'] = 7000
results = opt.solve(model,tee=True)
update_dual(pe,model)

Ipopt 3.12.8: print_user_options=yes
linear_solver=ma86
linear_system_scaling =mc19
linear_scaling_on_demand =no
max_iter=7000


List of user-set options:

                                    Name   Value                used
                linear_scaling_on_demand = no                    yes
                           linear_solver = ma86                  yes
                   linear_system_scaling = mc19                  yes
                                max_iter = 7000                  yes
                      print_user_options = yes                   yes

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.8, running with

  77r-4.8314997e+02 9.20e+00 1.00e+03   0.4 0.00e+00    -  0.00e+00 3.23e-07R  8
  78r-4.8314996e+02 7.66e+00 1.00e+03   0.4 2.56e+03    -  6.19e-03 5.43e-04f  1
  79r-4.8314991e+02 2.24e+00 1.00e+03   0.4 2.16e+03    -  8.94e-03 6.41e-03f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  80r-4.8314978e+02 2.22e+00 9.80e+02   0.4 2.42e+03    -  1.47e-02 1.68e-02f  1
  81r-4.8314959e+02 2.21e+00 1.72e+03   0.4 2.42e+03    -  8.57e-02 2.28e-02f  1
  82r-4.8314910e+02 2.29e+00 2.20e+03   0.4 2.16e+03    -  8.02e-02 5.51e-02f  1
  83r-4.8314873e+02 2.42e+00 7.05e+03   0.4 1.98e+03    -  1.50e-01 3.80e-02f  1
  84r-4.8314738e+02 2.80e+00 8.37e+03   0.4 2.16e+03    -  1.62e-01 1.15e-01f  1
  85r-4.8314674e+02 2.69e+00 2.70e+04   0.4 1.51e+03    -  4.62e-01 4.58e-02f  1
  86r-4.8314070e+02 3.77e+00 2.56e+04   0.4 1.25e+03    -  3.93e-01 2.29e-01f  1
  87r-4.8313630e+02 3.59e+00 2.69e+04   0.4 9.92e+02    -  2.88e-01 9.75e-02f  1
  88r-4.8312147e+02 3.58e+00

 177 -4.8309219e+02 1.85e-01 4.15e+04  -1.0 2.86e+04    -  4.09e-02 3.96e-03f  5
 178 -4.8309384e+02 2.67e-01 5.81e+04  -1.0 2.88e+04    -  5.64e-02 2.64e-03f  6
 179 -4.8309460e+02 2.91e-01 6.44e+04  -1.0 2.88e+04    -  6.61e-02 1.45e-03h  7
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 180 -4.8309461e+02 2.91e-01 6.58e+04  -1.0 2.88e+04    -  8.46e-02 2.39e-05h 13
 181 -4.8309477e+02 2.93e-01 8.03e+04  -1.0 2.92e+04    -  9.89e-02 3.86e-04h  9
 182 -4.8309484e+02 2.94e-01 1.32e+05  -1.0 2.96e+04    -  1.85e-01 1.98e-04h 10
 183 -4.8310943e+02 1.58e+02 4.89e+07  -1.0 3.10e+04    -  5.01e-02 1.07e-01w  1
 184 -4.8310924e+02 1.75e+02 8.94e+06  -1.0 8.84e+03    -  1.11e-02 2.41e-01w  1
 185 -4.8310922e+02 1.73e+02 8.82e+06  -1.0 2.15e+03    -  1.47e-01 1.26e-02w  1
 186 -4.8309495e+02 3.03e-01 1.42e+05  -1.0 1.02e+03    -  5.01e-02 8.33e-04h  7
 187r-4.8309495e+02 3.03e-01 1.00e+03  -0.7 0.00e+00    -  0.00e+00 4.17e-07R 19
 188r-4.8309495e+02 7.52e-03

 289 -4.8314996e+02 6.34e-02 4.38e+05  -1.0 3.41e+04    -  7.70e-03 1.53e-04f 10
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 290 -4.8314989e+02 1.02e+02 4.85e+07  -1.0 3.40e+04    -  2.83e-02 7.85e-02f  1
 291 -4.8314968e+02 1.64e+02 5.85e+07  -1.0 1.02e+04    -  8.38e-03 2.57e-01f  1
 292 -4.8314967e+02 1.62e+02 5.78e+07  -1.0 2.82e+03    -  2.90e-01 1.17e-02h  1
 293 -4.8314967e+02 1.62e+02 5.78e+07  -1.0 2.74e+03    -  4.33e-03 1.21e-04h  1
 294 -4.8314973e+02 1.62e+02 5.78e+07  -1.0 5.16e+02    -  1.76e-05 1.56e-05h  1
 295 -4.8314983e+02 1.62e+02 5.78e+07  -1.0 7.16e+02    -  1.53e-02 1.79e-05h  1
 296 -4.8315000e+02 1.62e+02 5.78e+07  -1.0 1.30e+03    -  1.95e-05 1.69e-05h  1
 297r-4.8315000e+02 1.62e+02 1.00e+03   2.0 0.00e+00    -  0.00e+00 9.02e-08R  2
 298r-4.8314998e+02 1.32e+02 4.40e+04   2.0 1.02e+05    -  5.43e-04 1.90e-04f  1
 299 -4.8314998e+02 1.32e+02 1.18e+04  -1.0 2.18e+03    -  2.43e-02 1.02e-04h  1
iter    objective    inf_pr 

 381 -4.8308726e+02 1.21e-03 5.56e+09  -1.0 1.70e+03    -  1.00e+00 1.15e-03h  9
 382 -4.8308750e+02 1.26e-03 8.51e+09  -1.0 1.70e+03    -  2.26e-01 1.14e-03h  9
 383 -4.8308774e+02 1.30e-03 2.87e+10  -1.0 1.70e+03    -  1.00e+00 1.14e-03h  9
 384 -4.8308798e+02 1.34e-03 4.43e+10  -1.0 1.70e+03    -  2.30e-01 1.13e-03h  9
 385 -4.8308822e+02 1.38e-03 1.51e+11  -1.0 1.70e+03    -  1.00e+00 1.13e-03h  9
 386 -4.8308838e+02 1.42e-03 2.02e+11  -1.0 1.82e+03    -  2.31e-01 1.05e-03h  9
 387 -4.8308851e+02 1.46e-03 4.00e+11  -1.0 1.88e+03    -  1.00e+00 1.02e-03h  9
 388 -4.8310843e+02 2.77e+00 2.22e+14  -1.0 1.96e+03    -  2.31e-01 2.48e-01w  1
 389 -4.8310885e+02 2.77e+00 2.21e+14  -1.0 2.73e+02    -  4.33e-03 2.10e-03w  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 390 -4.8310983e+02 2.76e+00 2.21e+14  -1.0 4.68e+02    -  3.03e-01 3.18e-03w  1
 391 -4.8308858e+02 1.50e-03 4.27e+11  -1.0 1.54e+02    -  2.31e-01 9.70e-04h  8
 392 -4.8308867e+02 1.55e-03

 477 -4.8310640e+02 4.51e-03 1.16e+14  -1.0 1.63e+03    -  1.00e+00 1.26e-03h  8
 478 -4.8310673e+02 4.55e-03 1.16e+14  -1.0 1.63e+03    -  2.52e-01 1.26e-03h  8
 479 -4.8314957e+02 8.24e-01 5.69e+14  -1.0 1.64e+03    -  1.00e+00 1.62e-01w  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 480 -4.8314965e+02 8.72e-01 6.90e+14  -1.0 1.78e+03    -  4.79e-02 4.90e-02w  1
 481 -4.8314965e+02 8.60e-01 6.81e+14  -1.0 6.69e+01    -  9.68e-03 1.32e-02w  1
 482 -4.8310707e+02 4.60e-03 1.16e+14  -1.0 4.61e+01    -  1.00e+00 1.26e-03h  7
 483 -4.8310740e+02 4.64e-03 1.16e+14  -1.0 1.64e+03    -  2.48e-01 1.26e-03h  8
 484 -4.8310773e+02 4.68e-03 1.16e+14  -1.0 1.64e+03    -  1.00e+00 1.26e-03h  8
 485 -4.8310806e+02 4.73e-03 1.16e+14  -1.0 1.65e+03    -  2.43e-01 1.26e-03h  8
 486 -4.8310838e+02 4.77e-03 1.16e+14  -1.0 1.65e+03    -  1.00e+00 1.27e-03h  8
 487 -4.8310870e+02 4.82e-03 1.16e+14  -1.0 1.65e+03    -  2.39e-01 1.26e-03h  8
 488 -4.8310902e+02 4.86e-03

 571 -4.8314195e+02 5.12e-01 4.72e+14  -1.0 1.58e+02    -  7.54e-03 9.90e-03w  1
 572 -4.8314183e+02 4.97e-01 4.50e+14  -1.0 4.78e+01    -  2.44e-02 2.93e-02w  1
 573 -4.8312366e+02 7.44e-03 1.17e+14  -1.0 4.54e+01    -  1.00e+00 9.47e-04h  7
 574 -4.8312380e+02 7.46e-03 1.17e+14  -1.0 1.73e+03    -  1.31e-01 9.40e-04h  8
 575 -4.8312394e+02 7.49e-03 1.17e+14  -1.0 1.73e+03    -  1.00e+00 9.33e-04h  8
 576 -4.8312407e+02 7.51e-03 1.17e+14  -1.0 1.73e+03    -  1.29e-01 9.26e-04h  8
 577 -4.8312421e+02 7.53e-03 1.17e+14  -1.0 1.73e+03    -  1.00e+00 9.19e-04h  8
 578 -4.8312434e+02 7.55e-03 1.17e+14  -1.0 1.73e+03    -  1.27e-01 9.13e-04h  8
 579 -4.8312447e+02 7.58e-03 1.17e+14  -1.0 1.73e+03    -  1.00e+00 9.06e-04h  8
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 580 -4.8312460e+02 7.60e-03 1.17e+14  -1.0 1.72e+03    -  1.25e-01 9.00e-04h  8
 581 -4.8312473e+02 7.62e-03 1.17e+14  -1.0 1.72e+03    -  1.00e+00 8.93e-04h  8
 582 -4.8312486e+02 7.64e-03

 666 -4.8313309e+02 1.01e-02 1.19e+14  -1.0 1.43e+03    -  1.00e+00 1.52e-03h  6
 667 -4.8313316e+02 1.02e-02 1.19e+14  -1.0 1.41e+03    -  4.91e-02 1.49e-03h  6
 668 -4.8313321e+02 1.02e-02 1.19e+14  -1.0 1.40e+03    -  4.81e-01 1.46e-03h  6
 669 -4.8313326e+02 1.02e-02 1.19e+14  -1.0 1.38e+03    -  5.06e-02 1.43e-03h  6
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 670 -4.8313329e+02 1.02e-02 1.19e+14  -1.0 1.36e+03    -  1.00e+00 1.40e-03h  6
 671 -4.8313332e+02 1.03e-02 1.19e+14  -1.0 1.35e+03    -  4.54e-02 1.38e-03h  6
 672 -4.8313334e+02 1.03e-02 1.19e+14  -1.0 1.33e+03    -  4.97e-01 1.35e-03h  6
 673 -4.8313335e+02 1.03e-02 1.20e+14  -1.0 1.31e+03    -  4.66e-02 1.33e-03h  6
 674 -4.8313340e+02 4.35e-02 1.23e+14  -1.0 1.29e+03    -  9.93e-01 4.18e-02w  1
 675 -4.8313647e+02 4.28e-02 1.21e+14  -1.0 4.74e+01    -  1.88e-02 1.64e-02w  1
 676 -4.8313568e+02 3.88e-02 1.13e+14  -1.0 4.66e+01    -  3.76e-02 9.32e-02w  1
 677 -4.8313335e+02 1.04e-02

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 760 -4.8314928e+02 8.52e-03 5.48e+12  -1.0 1.10e+01    -  5.58e-01 1.69e-02h  6
 761 -4.8314929e+02 8.49e-03 4.92e+12  -1.0 1.06e+01    -  1.00e+00 1.73e-02h  6
 762 -4.8314930e+02 8.45e-03 4.66e+12  -1.0 1.03e+01    -  5.80e-01 1.77e-02h  6
 763 -4.8314931e+02 8.41e-03 4.16e+12  -1.0 9.91e+00    -  1.00e+00 1.80e-02h  6
 764 -4.8314932e+02 8.37e-03 3.92e+12  -1.0 9.56e+00    -  6.04e-01 1.84e-02h  6
 765 -4.8314965e+02 1.21e-01 1.71e+12  -1.0 9.20e+00    -  1.00e+00 6.02e-01w  1
 766 -4.8314987e+02 1.40e-01 1.14e+14  -1.0 3.31e+00    -  3.25e-02 4.92e-01w  1
 767 -4.8314906e+02 1.40e-01 1.14e+14  -1.0 9.07e+01    -  1.01e-04 1.15e-03w  1
 768 -4.8314933e+02 8.33e-03 3.48e+12  -1.0 7.66e+00    -  1.00e+00 1.88e-02h  5
 769 -4.8314898e+02 3.81e-03 3.45e+12  -1.0 8.86e+00    -  1.25e-01 2.41e-01H  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 770 -4.8314898e+02 3.80e-03

In [22]:
model.Q_main.value

-8.632745760515922

In [23]:
model.T.value

483.15

In [24]:
# model.solutions.store_to(results)
# with open('../saved_solutions/reactive_flash_300C_n58.pickle','wb') as f:
#     pickle.dump(results,f)

In [ ]:
print('Component\t\tLiquid: {:.4f}\t\t\tVapor: {:.4f}\t\t\tlog K'.format(model.L['out'].value,model.V['out'].value))
print('-'*108)
for i in model.x:
    print('{:10s}'.format(i),'\t\t{:8.4%}\t\t\t{:8.4%}\t\t\t{:.4f}'.format(model.x[i].value,model.y[i].value,np.log10(model.y[i].value/model.x[i].value)))

# Iterative Solve for Data Analysis

In [ ]:
update_dual(pe,model)

In [ ]:
opt.options['warm_start_init_point'] = 'yes'
opt.options['warm_start_bound_push'] = 1e-20
opt.options['warm_start_mult_bound_push'] = 1e-20
opt.options['mu_init'] = 1e-6

In [ ]:
rf_data = {}
rf_data['r'] = {}; rf_data['T'] = []; rf_data['Q'] = []; rf_data['V'] = []; rf_data['L'] = []; 
rf_data['y_CO'] = []; rf_data['y_H2'] = []; rf_data['y_CO2'] = []; rf_data['y_H2O'] = []; rf_data['r_WGS'] = []; rf_data['r_FT'] = []
rf_data['f_V_CO'] = []; rf_data['f_V_H2'] = [];

for i in model.r_total_comp:
    rf_data['r'][i] = []
    
Trange = np.arange(300+273.15,199+273.15,-2)

for Tub in Trange:
    model.T.setub(Tub)
    results = opt.solve(model,tee=False)
    update_dual(pe,model)
    # print('-'*72)
    print('Solved, Solution T = {:.2f} K\t|\tV = {:.5f}\t|\tL = {:.5f}\t|'.format(model.T.value,model.V['out'].value,model.L['out'].value))
    # print('-'*72)
    rf_data['T'].append(model.T.value)
    rf_data['Q'].append(model.Q_main.value)
    rf_data['V'].append(model.V['out'].value)
    rf_data['L'].append(model.L['out'].value)    

    for i in model.r_total_comp:
        rf_data['r'][i].append(model.r_total_comp[i].value)
    
    
    rf_data['y_H2O'].append(model.y['H2O'].value)
    rf_data['y_CO'].append(model.y['CO'].value)
    rf_data['y_H2'].append(model.y['H2'].value)
    rf_data['y_CO2'].append(model.y['CO2'].value)
    rf_data['r_WGS'].append(model.kinetics_block.r_WGS.value)
    rf_data['r_FT'].append(model.kinetics_block.r_FT_total.value)
    
    rf_data['f_V_CO'].append(model.f_V['CO'].value)
    rf_data['f_V_H2'].append(model.f_V['H2'].value)

## Some Observations

* This system has multiple solution under same Q.
* The reaction rate peaks at ~530K, due to decreasing $\alpha$, increasing light component, uses up more feed.
* Note that when 'fully' utilizing catalyst, the system seems to always require cooling.

In [ ]:
fig, ax1 = plt.subplots(figsize=(16,9))
line1 = ax1.plot(rf_data['T'],rf_data['Q'], 'b-')
ax1.set_xlabel('Temperature (k)', color='K',fontsize=18)
ax1.set_ylabel('Q (MW)', color='b',fontsize=18)
ax1.tick_params('y', colors='b',labelsize=18)
ax1.tick_params('x', colors='k',labelsize=18)
ax1.grid()

ax2 = ax1.twinx()
line2 = ax2.plot(rf_data['T'],rf_data['r_FT'], 'r')
line3 = ax2.plot(rf_data['T'],rf_data['r_WGS'], 'r--')
line4 = ax2.plot(rf_data['T'],rf_data['V'], 'k')
line5 = ax2.plot(rf_data['T'],rf_data['L'], 'k--')
ax2.set_ylabel('$r_{FT}$: (kmol/s Carbon Number)', color='r',fontsize=18)
ax2.tick_params('y', colors='r',labelsize=18)

ax1.legend(line1+line2+line3+line4+line5,['Q','$r_{FT}$','$r_{WGS}$','V','L'],fontsize=18,loc=1)
ax1.set_title('Heat Removal Rate and Reaction Rate vs T',fontsize=18)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(16,9))
ax.plot(rf_data['T'],rf_data['y_CO'])
ax.plot(rf_data['T'],rf_data['y_H2'])
ax.plot(rf_data['T'],rf_data['y_CO2'])
ax.plot(rf_data['T'],rf_data['y_H2O'])

ax.legend(['$y_{CO}$','$y_{H2}$','$y_{CO2}$','$y_{H2O}$'],fontsize=18)
ax.set_title('Gas Phase Inorganic Composition vs T',fontsize=18)

ax.set_ylabel('Mole Fraction (%)', color='K',fontsize=18)
ax.set_xlabel('Temperature (k)', color='K',fontsize=18)
ax.tick_params('y', colors='k',labelsize=18)
ax.tick_params('x', colors='k',labelsize=18)
ax.grid()
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(16,9))
ax.plot(rf_data['T'],rf_data['r']['CO'])
ax.plot(rf_data['T'],rf_data['r']['H2'])
ax.plot(rf_data['T'],rf_data['r']['CO2'])
ax.plot(rf_data['T'],rf_data['r']['H2O'])

ax.legend(['$r_{CO}$','$r_{H2}$','$r_{CO2}$','$r_{H2O}$'],fontsize=18)
ax.set_title('Inorganic Reaction Rate vs T',fontsize=18)

ax.set_ylabel('Reaction Rate (kmol/s)', color='K',fontsize=18)
ax.set_xlabel('Temperature (k)', color='K',fontsize=18)
ax.tick_params('y', colors='k',labelsize=18)
ax.tick_params('x', colors='k',labelsize=18)
ax.grid()
plt.show()

## Product Distribution (mole)

In [ ]:
reaction_data = trans_product_mole(rf_data['r'])['unscaled']
reaction_data_scaled = trans_product_mole(rf_data['r'])['scaled']

In [ ]:
fig, ax = plt.subplots(figsize=(16,9))
ax.stackplot(rf_data['T'], reaction_data_scaled['c1'], reaction_data_scaled['c2'],reaction_data_scaled['c3'],reaction_data_scaled['c4'],\
            reaction_data_scaled['napha'],reaction_data_scaled['gasoline'],reaction_data_scaled['diesel'],reaction_data_scaled['heavy'])
plt.legend(['C1','C2','C3','C4','Napha','gasoline','diesel','heavy'])
ax.set_title('Product Composition (Mole) vs T',fontsize=18)
ax.set_ylabel('Mole Fraction (%)', color='K',fontsize=18)
ax.set_xlabel('Temperature (k)', color='K',fontsize=18)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(16,9))
ax.stackplot(rf_data['T'], reaction_data['c1'], reaction_data['c2'],reaction_data['c3'],reaction_data['c4'],\
            reaction_data['napha'],reaction_data['gasoline'],reaction_data['diesel'],reaction_data['heavy'])
plt.legend(['C1','C2','C3','C4','Napha','gasoline','diesel','heavy'])
ax.set_title('Total Production Rate (Mole) vs T',fontsize=18)
ax.set_ylabel('Reaction Rate (kmol/s)', color='K',fontsize=18)
ax.set_xlabel('Temperature (k)', color='K',fontsize=18)
plt.show()

## Product Distribution (mass)

In [ ]:
mass_reaction_data = trans_product_mass(rf_data['r'])['unscaled']
mass_reaction_scaled = trans_product_mass(rf_data['r'])['scaled']

In [ ]:
fig, ax = plt.subplots(figsize=(16,9))
ax.stackplot(rf_data['T'], mass_reaction_scaled['c1'], mass_reaction_scaled['c2'],mass_reaction_scaled['c3'],mass_reaction_scaled['c4'],\
            mass_reaction_scaled['napha'],mass_reaction_scaled['gasoline'],mass_reaction_scaled['diesel'],mass_reaction_scaled['heavy'])
plt.legend(['C1','C2','C3','C4','Napha','gasoline','diesel','heavy'])
ax.set_title('Product Composition (Mass) vs T',fontsize=18)
ax.set_ylabel('Mass Fraction (%)', color='K',fontsize=18)
ax.set_xlabel('Temperature (k)', color='K',fontsize=18)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(16,9))
ax.stackplot(rf_data['T'], mass_reaction_data['c1'], mass_reaction_data['c2'],mass_reaction_data['c3'],mass_reaction_data['c4'],\
            mass_reaction_data['napha'],mass_reaction_data['gasoline'],mass_reaction_data['diesel'],mass_reaction_data['heavy'])
plt.legend(['C1','C2','C3','C4','Napha','gasoline','diesel','heavy'])
ax.set_title('Total Production Rate (mass) vs T',fontsize=18)
ax.set_ylabel('Reaction Rate (kg/s)', color='K',fontsize=18)
ax.set_xlabel('Temperature (k)', color='K',fontsize=18)
plt.show()

In [ ]:
# model.solutions.store_to(results)
# with open('../saved_solutions/reactive_flash_200C_n58.pickle','wb') as f:
#     pickle.dump(results,f)

# VLE Validation using AspenPlus

### Feed (Reactor Effluent)

In [ ]:
print('Temperature (C): \t{:.2f}\t\t'.format(model.T.value-273.15))
print('Conversion(H2+CO): \t{:.2f}\t\t'.format(1-((model.y['CO'].value+model.y['H2'].value)*model.V['out'].value)))
print('Total FLow (kmol/s): \t{:.2f}\t\t'.format(model.L['out'].value+model.V['out'].value))
print('-'*72)
for i in model.x:
    print('{:10s}'.format(i),'\t\t{:.4%}'.format((model.x[i].value*model.L['out'].value\
        +model.y[i].value*model.V['out'].value)/(model.L['out'].value+model.V['out'].value)))

### Phase Separation

In [ ]:
print('Component\t\tLiquid: {:.4f}\t\t\tVapor: {:.4f}\t\t\tlog K'.format(model.L['out'].value,model.V['out'].value))
print('-'*108)
for i in model.x:
    print('{:10s}'.format(i),'\t\t{:8.4%}\t\t\t{:8.4%}\t\t\t{:.4f}'.format(model.x[i].value,model.y[i].value,np.log10(model.y[i].value/model.x[i].value)))